In [ ]:
#importação de dados
import os
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [ ]:
#tratando os outliers

def tratar_outliers(df, colunas = None):
  """
  trata outliers para colunas numéricas

  parâmetros:
  df: dataframe a ser inspecionado
  colunas: lista de colunas ou None (aplica em todas as numéricas)

  """
  if colunas is None:
    colunas = df.select_dtypes(include = ['number']).columns

  for col in colunas:
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr

    df[col] = np.where(
        (df[col] < limite_inferior) | (df[col] > limite_superior),
        np.nan,
        df[col]
    )
  return df

#extraindo o mes do dataset

def extrai_mes(df):
    df = df.copy()
    df["mes"] = pd.to_datetime(df["DATA"]).dt.month  # Extrai mês diretamente
    return df

#gerando as metricas mensais
def gerar_metricas_mensais(df):
    try:
        #lista de colunas obrigatórias
        required_columns = [
          "PRECIPITAÇÃO TOTAL, HORÁRIO (MM)",  # MM em maiúsculo
          "PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (MB)",  # MB em maiúsculo
          "RADIACAO GLOBAL (KJ/M²)",  # KJ e M² em maiúsculo
          "TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",
          "TEMPERATURA DO PONTO DE ORVALHO (°C)",
          "UMIDADE RELATIVA DO AR, HORARIA (%)",
          "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)",
          "VENTO, VELOCIDADE HORARIA (M/S)"  # M/S em maiúsculo
        ]

        #criar colunas faltantes com valores NaN
        for col in required_columns:
            if col not in df.columns:
                df[col] = np.nan
                print(f"Aviso: Coluna {col} ausente. Preenchida com NaN.")

        #processamento normal
        df['date'] = pd.to_datetime(df['DATA'])
        df_agg = df.groupby(df['date'].dt.month).agg(
            cli_precip_dia_med=("PRECIPITAÇÃO TOTAL, HORÁRIO (MM)", lambda x: x.notna().sum()),
            cli_precip_total=("PRECIPITAÇÃO TOTAL, HORÁRIO (MM)", "sum"),
            cli_pressao_atm_med=("PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (MB)", "mean"),
            cli_rad_global_med=("RADIACAO GLOBAL (KJ/M²)", "mean"),
            cli_temp_ar_med=("TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)", "mean"),
            cli_temp_orvalho_med=("TEMPERATURA DO PONTO DE ORVALHO (°C)", "mean"),
            cli_umid_rel_ar_med=("UMIDADE RELATIVA DO AR, HORARIA (%)", "mean"),
            cli_umid_rel_min_med=("UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)", "mean"),
            cli_umid_rel_min_max=("UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)", "max"),
            cli_umid_rel_min_min=("UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)", "min"),
            cli_vento_veloc_max=("VENTO, VELOCIDADE HORARIA (M/S)", "max"),
            cli_vento_veloc_med=("VENTO, VELOCIDADE HORARIA (M/S)", "mean")
        ).reset_index().rename(columns={'date': 'mes'})

        #cálculo seguro da precipitação média
        df_agg["cli_precip_dia_med"] = df_agg["cli_precip_total"] / df_agg["cli_precip_dia_med"].replace(0, np.nan)

        #renomeação final
        df_agg.rename(columns={
            "cli_precip_dia_med": "Precipitação Média (MM)",
            "cli_precip_total": "Precipitação Total (MM)",
            "cli_pressao_atm_med": "Pressão Média (MB)",
            "cli_rad_global_med": "Radiação Média (KJ/M²)",
            "cli_temp_ar_med": "Temperatura Média (°C)",
            "cli_temp_orvalho_med": "Temperatura Orvalho Média (°C)",
            "cli_umid_rel_ar_med": "Umidade Média (%)",
            "cli_umid_rel_min_med" : "Umidade Mínima Média (%)",
            "cli_umid_rel_min_max" : "Umidade Mínima Máxima (%)",
            "cli_umid_rel_min_min" : "Umidade Mínima Mínima (%)",
            "cli_vento_veloc_max" : "Velocidade Máxima do Vento (M/S)",
            "cli_vento_veloc_med" : "Velocidade Média do Vento (M/S)"
        }, inplace=True)

        return df_agg

    except Exception as e:
        print(f"Erro na agregação: {str(e)}")
        return None

def ler_arquivo(nome_arquivo):
    try:
        #tentar ler com UTF-8
        df = pd.read_csv(nome_arquivo, sep=';', skiprows=8, decimal=',', encoding='utf-8')
    except UnicodeDecodeError:
        #tentar Latin-1 como fallback
        df = pd.read_csv(nome_arquivo, sep=';', skiprows=8, decimal=',', encoding='latin-1')

    #debug: Mostrar colunas originais
    #print(f"\nArquivo: {os.path.basename(nome_arquivo)}")
    #print("Colunas originais:", df.columns.tolist())


    #mapeamento de variações de colunas de data
    data_column_variants = [
        "Data",
        "DATA (YYYY-MM-DD)",
        "DATA",
        "DATA_MEDICAO",
        "DIA"
    ]
    df.columns = df.columns.str.upper().str.strip()
    #renomear para "DATA" se alguma variação existir
    for col in data_column_variants:
      if col in df.columns:
        df.rename(columns={col: "DATA"}, inplace=True)
        #print(f"Coluna de data padronizada: {col} -> DATA")
        break
    else:  #fora do loop
      available_cols = df.columns.tolist()
      print(f"Colunas disponíveis: {available_cols}")
      raise ValueError(f"Nenhuma coluna de data reconhecida em {nome_arquivo}")

    #print("Colunas após renomeação:", df.columns.tolist())
    return df

In [ ]:
#compactando cada csv de cada estação em 12 linhas, 1 para cada mês
drive.mount('/content/drive')

ESTADOS = {
    'AC': 'Acre', 'AM': 'Amazonas', 'AP': 'Amapá', 'PA': 'Pará',
    'RO': 'Rondônia', 'RR': 'Roraima', 'TO': 'Tocantins'
}

def processar_arquivos_historico(base_path):
    """
    Processa arquivos CSV brutos e salva um arquivo por estado/ano
    """
    caminho_saida = '/content/drive/MyDrive/projCDat_25_1/datasets/cooked/INMET/tentativa04'
    os.makedirs(caminho_saida, exist_ok=True)

    #dicionário para acumular dados por estado/ano
    dados_por_estado_ano = {}

    pastas_ano = sorted(
        [pasta for pasta in os.listdir(base_path)
         if os.path.isdir(os.path.join(base_path, pasta))
         and pasta.isdigit()
         and 2000 <= int(pasta) <= 2023],
        key=int
    )

    for pasta in pastas_ano:
        ano = int(pasta)
        caminho_pasta_ano = os.path.join(base_path, pasta)

        print(f"\nProcessando ano: {ano} | Caminho: {caminho_pasta_ano}")

        for arquivo in os.listdir(caminho_pasta_ano):
            if not arquivo.upper().endswith('.CSV'):
                continue

            try:
                caminho_completo = os.path.join(caminho_pasta_ano, arquivo)
                df = ler_arquivo(caminho_completo)
                df = tratar_outliers(df)
                df = extrai_mes(df)

                #extrair metadados
                partes = arquivo.split('_')
                sigla_estado = partes[2]
                nome_estacao = ' '.join(partes[4:-2])

                #adicionar colunas
                df['estado'] = ESTADOS.get(sigla_estado, 'Desconhecido')
                df['ano'] = ano
                df['estacao'] = nome_estacao  # Mantemos a estação como coluna

                #gerar métricas mensais
                df_agg = gerar_metricas_mensais(df)

                #agrupar por estado/ano (chave do dicionário)
                chave = (sigla_estado, ano)
                if chave not in dados_por_estado_ano:
                    dados_por_estado_ano[chave] = []
                dados_por_estado_ano[chave].append(df_agg)

            except Exception as e:
                print(f"Erro no arquivo {arquivo}: {str(e)}")
                continue

    #salvar arquivos consolidados por estado/ano
    for (sigla_estado, ano), dataframes in dados_por_estado_ano.items():
        try:
            df_consolidado = pd.concat(dataframes, ignore_index=True)
            nome_estado = ESTADOS.get(sigla_estado, 'Desconhecido').replace(' ', '_')
            nome_saida = f"INMET_PROCESSADO_{sigla_estado}_{nome_estado}_{ano}.csv"
            caminho_final = os.path.join(caminho_saida, nome_saida)
            df_consolidado.to_csv(caminho_final, index=False)
            print(f"Arquivo salvo: {caminho_final}")

        except Exception as e:
            print(f"Erro ao salvar {sigla_estado}/{ano}: {str(e)}")

    print("\nProcessamento concluído!")
BASE_PATH = '/content/drive/MyDrive/projCDat_25_1/datasets/raw/INMET/brutos'

df_final = processar_arquivos_historico(BASE_PATH)

if not df_final.empty:
  caminho_saida = '/content/drive/MyDrive/projCDat_25_1/datasets/cooked/INMET/tentativa04'
  df_final.to_csv(caminho_saida, index=False)
  print(f"Dataset consolidado salvo em: {caminho_saida}")
else:
  print("Nenhum arquivo foi processado.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Processando ano: 2000 | Caminho: /content/drive/MyDrive/projCDat_25_1/datasets/raw/INMET/brutos/2000

Processando ano: 2001 | Caminho: /content/drive/MyDrive/projCDat_25_1/datasets/raw/INMET/brutos/2001

Processando ano: 2002 | Caminho: /content/drive/MyDrive/projCDat_25_1/datasets/raw/INMET/brutos/2002

Processando ano: 2003 | Caminho: /content/drive/MyDrive/projCDat_25_1/datasets/raw/INMET/brutos/2003

Processando ano: 2004 | Caminho: /content/drive/MyDrive/projCDat_25_1/datasets/raw/INMET/brutos/2004

Processando ano: 2005 | Caminho: /content/drive/MyDrive/projCDat_25_1/datasets/raw/INMET/brutos/2005

Processando ano: 2006 | Caminho: /content/drive/MyDrive/projCDat_25_1/datasets/raw/INMET/brutos/2006

Processando ano: 2007 | Caminho: /content/drive/MyDrive/projCDat_25_1/datasets/raw/INMET/brutos/2007

Processando ano: 2008 | Caminho: /content/drive/MyDriv

AttributeError: 'NoneType' object has no attribute 'empty'

In [ ]:
#unificação dos datasets de clima
drive.mount('/content/drive')

#configurações iniciais
path = '/content/drive/MyDrive/projCDat_25_1/datasets/cooked/INMET/tentativa04'
estados = {
    'AC': 'Acre',
    'AM': 'Amazonas',
    'AP': 'Amapá',
    'PA': 'Pará',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'TO': 'Tocantins'
}

#listar arquivos CSV
arquivos = [f for f in os.listdir(path) if f.endswith('.csv')]

#lista para armazenar DataFrames
dataframes = []

for arquivo in arquivos:
    try:
        #extrair informações do nome do arquivo
        partes = arquivo.split('_')
        sigla_estado = partes[2]
        ano = partes[-1].split('.')[0]

        #verificar se é um arquivo válido
        if sigla_estado not in estados or not ano.isdigit():
            continue

        #carregar o CSV
        df = pd.read_csv(os.path.join(path, arquivo))

        #adicionar colunas de ano e estado
        df['ano'] = int(ano)
        df['estado'] = estados[sigla_estado]

        #lista de colunas para tratamento (todas exceto ano e estado)
        colunas_numericas = df.columns.drop(['mes', 'ano', 'estado'])

        #substituir valores negativos por NaN
        for coluna in colunas_numericas:
            if df[coluna].dtype in [np.int64, np.float64]:
                df[coluna] = df[coluna].where(df[coluna] >= 0, np.nan)

        dataframes.append(df)

    except Exception as e:
        print(f"Erro ao processar {arquivo}: {str(e)}")

#combinar todos os DataFrames
df_final = pd.concat(dataframes, ignore_index=True)

#reordenar colunas
colunas = ['estado', 'ano', 'mes'] + [c for c in df_final.columns if c not in ['estado', 'ano', 'mes']]
df_final = df_final[colunas]

#salvar arquivo combinado
caminho_saida = os.path.join(path, 'DADOS_COMBINADOS(sem_outliers).csv')
df_final.to_csv(caminho_saida, index=False)

print(f"Processo concluído! Arquivo salvo em: {caminho_saida}")
print(f"Total de registros: {len(df_final)}")
print(f"Colunas: {', '.join(df_final.columns)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processo concluído! Arquivo salvo em: /content/drive/MyDrive/projCDat_25_1/datasets/cooked/INMET/tentativa04/DADOS_COMBINADOS(sem_outliers).csv
Total de registros: 14257
Colunas: estado, ano, mes, Precipitação Média (MM), Precipitação Total (MM), Pressão Média (MB), Radiação Média (KJ/M²), Temperatura Média (°C), Temperatura Orvalho Média (°C), Umidade Média (%), Umidade Mínima Média (%), Umidade Mínima Máxima (%), Umidade Mínima Mínima (%), Velocidade Máxima do Vento (M/S), Velocidade Média do Vento (M/S)


In [ ]:
#fazendo o megazord

#montar o Google Drive
drive.mount('/content/drive')

#carregar os datasets
caminho_meteorologia = '/content/drive/MyDrive/projCDat_25_1/datasets/cooked/INMET/DADOS_COMBINADOS.csv'
caminho_mapbiomas = '/content/drive/MyDrive/projCDat_25_1/datasets/cooked/MapBiomas_Juncao/juncao_mapBiomas.csv'

df_met = pd.read_csv(caminho_meteorologia)
df_map = pd.read_csv(caminho_mapbiomas)

estados = {
    'AC': 'Acre',
    'AM': 'Amazonas',
    'AP': 'Amapá',
    'PA': 'Pará',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'TO': 'Tocantins'
}

#renomear colunas do MapBiomas (incluindo a nova alteração)
df_map = df_map.rename(columns={
    '_ano': 'ano',
    '_mes': 'mes',
    '_estado': 'estado',
    'que_area_queimada': 'Quantidade de área queimada (HA)'
})

#restante do código mantido igual
df_map = df_map.astype({'ano': int, 'mes': int})
df_met = df_met.astype({'ano': int, 'mes': int})


#padronizar nomes dos estados no MapBiomas (caso use siglas)
df_map['estado'] = df_map['estado'].str.upper().map(
    lambda x: estados.get(x, x) if x in estados.keys() else x
)

#fazer o merge com a coluna renomeada
df_final = pd.merge(
    df_met,
    df_map[['ano', 'mes', 'estado', 'Quantidade de área queimada (HA)']],  # Usar novo nome
    on=['ano', 'mes', 'estado'],
    how='inner'
)

#salvar o resultado (caminho mantido)
caminho_final = '/content/drive/MyDrive/projCDat_25_1/datasets/cooked/INMET/preprocessed.csv'
df_final.to_csv(caminho_final, index=False)

print("Coluna renomeada com sucesso! Confira o resultado:")
display(df_final.head(3))

# Tratamento de dados faltantes e de _outliers_
Tratamos os outliers após compararmos os resultados dos modelos com e sem o tratamento.
Mantendo os outliers, observamos valores incoerentes com a realidade, como temperaturas médias em torno de 12°C,
além de um aumento significativo na quantidade de dados faltantes.
Em relação aos valores ausentes, optamos por remover a coluna de radiação,
que apresentava mais de 70% dos dados faltantes. Consideramos inviável aplicar técnicas de imputação,
tanto pela grande quantidade de valores ausentes quanto pela baixa relevância da radiação para as queimadas na Amazônia.
A região apresenta alta umidade ao longo do ano, o que dificulta a ocorrência de queimadas por causas naturais, como o acúmulo de radiação solar.
Além disso, dados do INPE indicam que a maioria das queimadas na região é provocada por ações humanas, o que reforça a decisão de remover esse atributo de forma justificada e eficaz.